In [1]:
import argparse
import json
import os
import pandas as pd
import os

import datetime
import numpy as np
from cgms_data_seg_t1dexi import CGMSDataSeg
from cnn_ohio import regressor, regressor_transfer, test_ckpt
from data_reader_T1DEXI import DataReader

In [2]:
import tensorflow as tf

# New method in TensorFlow 2.x:
# This will list the devices TensorFlow recognizes
print("TensorFlow version:", tf.__version__)
print("List of devices available to TensorFlow:")
print(tf.config.list_physical_devices())

TensorFlow version: 2.10.1
List of devices available to TensorFlow:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:

# Define the directory path
# directory_path = r'C:\Users\baiyi\OneDrive\Desktop\BGprediction\LB_split'  # Use a raw string for paths on Windows

# # List files without their extensions
# file_names = [os.path.splitext(file)[0] for file in os.listdir(directory_path)
#               if os.path.isfile(os.path.join(directory_path, file))]

# # Print the list of file names
# print(file_names)

['1', '1000', '1004', '1010', '1012', '1013', '1014', '1015', '1016', '1020', '1021', '1022', '1024', '103', '1033', '1039', '104', '1043', '1046', '1050', '1051', '1067', '107', '1072', '1076', '1077', '1082', '1090', '1093', '11', '110', '1100', '1103', '1107', '1110', '1112', '1115', '1118', '1119', '1120', '1121', '1123', '1127', '1138', '1139', '114', '1141', '1142', '1143', '1146', '1149', '115', '1152', '1155', '1156', '1157', '1158', '1161', '1163', '1170', '1171', '1185', '1193', '1194', '1195', '1199', '1201', '1202', '1203', '1205', '1206', '1210', '1211', '1219', '1221', '1224', '1230', '1239', '1243', '1247', '1248', '1249', '1250', '1253', '1255', '1257', '1261', '1265', '1266', '127', '1271', '1273', '1281', '1283', '1286', '1287', '1291', '1292', '1297', '1302', '1303', '1304', '1305', '1307', '1311', '1312', '1322', '1323', '1325', '1327', '1328', '1329', '1330', '1335', '1336', '1338', '1339', '1343', '1344', '1345', '1348', '1351', '1354', '1357', '1359', '1361', '13

In [3]:
def read_preprocess_cgm_data(subj):
    subject = pd.read_csv(f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/{subj}.csv')
    subject_cgm = subject.loc[subject['LBCAT'] == 'CGM']
    # Convert the 'LBDTC' column to datetime
    subject_cgm['LBDTC'] = pd.to_datetime(subject_cgm['LBDTC'])
    # Extract the date part
    subject_cgm['date'] = subject_cgm['LBDTC'].dt.date
    return subject_cgm
    # Count the number of occurrences of each unique date

In [8]:
# # Check the continuity of the selected items
# def calendar_array(dates, data):
#     i, j = zip(*[d.isocalendar()[1:] for d in dates])
#     i = np.array(i) - min(i)
#     j = np.array(j) - 1
#     ni = max(i) + 1

#     calendar = np.nan * np.zeros((ni, 7))
#     calendar[i, j] = data
#     return i, j, calendar


# def calendar_heatmap(ax, dates, data):
#     i, j, calendar = calendar_array(dates, data)
#     im = ax.imshow(calendar, interpolation='none', cmap='summer')
#     label_days(ax, dates, i, j, calendar)
#     label_months(ax, dates, i, j, calendar)
    
#     # Adjust the colorbar
#     cbar = ax.figure.colorbar(im, ax=ax, fraction=0.02, pad=0.04)
#     cbar.ax.tick_params(labelsize=8)  # Adjust colorbar tick label size if needed

# def label_days(ax, dates, i, j, calendar):
#     ni, nj = calendar.shape
#     day_of_month = np.nan * np.zeros((ni, 7))
#     day_of_month[i, j] = [d.day for d in dates]

#     for (i, j), day in np.ndenumerate(day_of_month):
#         if np.isfinite(day):
#             ax.text(j, i, int(day), ha='center', va='center')

#     ax.set(xticks=np.arange(7), 
#            xticklabels=['M', 'T', 'W', 'R', 'F', 'S', 'S'])
#     ax.xaxis.tick_top()

# def label_months(ax, dates, i, j, calendar):
#     month_labels = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
#                              'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
#     months = np.array([d.month for d in dates])
#     uniq_months = sorted(set(months))
#     yticks = [i[months == m].mean() for m in uniq_months]
#     labels = [month_labels[m - 1] for m in uniq_months]
#     ax.set(yticks=yticks)
#     ax.set_yticklabels(labels, rotation=90)



In [ ]:
# import datetime as dt
# import matplotlib.pyplot as plt
# import numpy as np
# # Look at the contiunity of the selected subjects, no need to run

# for d in selected_subjects:
    
#     subject_cgm = read_preprocess_cgm_data(d)

#     date_counts = subject_cgm['date'].value_counts().sort_index()
#     dates, data = date_counts.index, date_counts.values
#     fig, ax = plt.subplots(figsize=(6, 10))
#     calendar_heatmap(ax, dates, data)
#     plt.show()

In [4]:
overlap = ['854',
 '979',
 '816',
 '953',
 '981',
 '1617',
 '1343',
 '987',
 '255',
 '907',
 '856',
 '354',
 '894',
 '862',
 '900',
 '695']

In [3]:
# For population
# Extract those samples from dataset

# for subj in overlap:
#     subject = pd.read_csv(f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/{subj}.csv')
#     subject_cgm = subject.loc[subject['LBCAT'] == 'CGM']
    
#     # split_index = int(len(subject_cgm) * 0.8)
#     # Split the DataFrame
#     train_df = subject


#     # Save the DataFrames to CSV files
#     train_df.to_csv(f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/population_training/{subj}_training_data.csv', index=False)
#     # test_df.to_csv(f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/testing/{subj}_testing_data.csv', index=False)

#     # Optionally, confirm the operation
#     print(f"Training data saved with {len(train_df)} records.")
#     # print(f"Testing data saved with {len(test_df)} records.")

Training data saved with 7870 records.
Training data saved with 7930 records.
Training data saved with 7606 records.
Training data saved with 7632 records.
Training data saved with 2880 records.
Training data saved with 7972 records.
Training data saved with 7603 records.
Training data saved with 7372 records.
Training data saved with 7503 records.
Training data saved with 7475 records.
Training data saved with 7945 records.
Training data saved with 8011 records.
Training data saved with 7855 records.
Training data saved with 7537 records.
Training data saved with 5611 records.
Training data saved with 7963 records.
Training data saved with 7965 records.
Training data saved with 7969 records.


In [7]:
# Divide training and testing
# Generate folder for training and test data

for subj in overlap:
    subject = pd.read_csv(f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/{subj}.csv')
    subject_cgm = subject.loc[subject['LBCAT'] == 'CGM']
    
    split_index = int(len(subject_cgm) * 0.8)
    # Split the DataFrame
    train_df = subject[:split_index]
    test_df = subject[split_index:]

    # Save the DataFrames to CSV files
    train_df.to_csv(f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/training/{subj}_training_data.csv', index=False)
    test_df.to_csv(f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/testing/{subj}_testing_data.csv', index=False)

    # Optionally, confirm the operation
    print(f"Training data saved with {len(train_df)} records.")
    print(f"Testing data saved with {len(test_df)} records.")

Training data saved with 6295 records.
Testing data saved with 1575 records.
Training data saved with 6343 records.
Testing data saved with 1587 records.
Training data saved with 6084 records.
Testing data saved with 1522 records.
Training data saved with 6104 records.
Testing data saved with 1528 records.
Training data saved with 2303 records.
Testing data saved with 577 records.
Training data saved with 6376 records.
Testing data saved with 1596 records.
Training data saved with 6081 records.
Testing data saved with 1522 records.
Training data saved with 5896 records.
Testing data saved with 1476 records.
Training data saved with 6001 records.
Testing data saved with 1502 records.
Training data saved with 5979 records.
Testing data saved with 1496 records.
Training data saved with 6355 records.
Testing data saved with 1590 records.
Training data saved with 6408 records.
Testing data saved with 1603 records.
Training data saved with 6283 records.
Testing data saved with 1572 records.


In [8]:
def read_preprocess_cgm_training(subj):
    subject_cgm = pd.read_csv(f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/training/{subj}_training_data.csv')

    subject_cgm['LBDTC'] = pd.to_datetime(subject_cgm['LBDTC'])
    # Extract the date part
    subject_cgm['date'] = subject_cgm['LBDTC'].dt.date
    subject_cgm_concise = subject_cgm[["LBORRES", "LBDTC"]]
    return subject_cgm_concise
    

In [9]:
# An example 
subject_cgm = read_preprocess_cgm_training(overlap[1])

In [10]:
subject_cgm

,LBORRES,LBDTC
0,71.0,2021-03-12 00:02:14
1,69.0,2021-03-12 00:07:13
2,67.0,2021-03-12 00:12:13
3,66.0,2021-03-12 00:17:13
4,65.0,2021-03-12 00:22:13
...,...,...
6338,83.0,2021-04-03 08:48:24
6339,83.0,2021-04-03 08:53:24
6340,83.0,2021-04-03 08:58:24
6341,84.0,2021-04-03 09:03:24


In [31]:
import datetime
# Segment the data into several pieces
# Assuming self.interval_timedelta is set, for example:
interval_timedelta = datetime.timedelta(minutes=6)  # Example timedelta of 6 minutes, providing a range for latency

# Create a list to store the results
res = []

# Initialize the first group
if not subject_cgm.empty:
    current_group = [subject_cgm.iloc[0]['LBORRES']]
    last_time = subject_cgm.iloc[0]['LBDTC']

# Iterate over rows in DataFrame starting from the second row
for index, row in subject_cgm.iloc[1:].iterrows():
    current_time = row['LBDTC']
    if (current_time - last_time) <= interval_timedelta:
        # If the time difference is within the limit, add to the current group
        current_group.append(row['LBORRES'])
    else:
        # Otherwise, start a new group
        res.append(current_group)
        current_group = [row['LBORRES']]
    last_time = current_time

# Add the last group if it's not empty
if current_group:
    res.append(current_group)

# Functions

In [5]:
def read_preprocess_cgm_training(subj):
    subject_cgm = pd.read_csv(subj)

    subject_cgm['LBDTC'] = pd.to_datetime(subject_cgm['LBDTC'])
    # Extract the date part
    subject_cgm['date'] = subject_cgm['LBDTC'].dt.date
    subject_cgm_concise = subject_cgm[["LBORRES", "LBDTC"]]
    return subject_cgm_concise
    

In [6]:
def preporcess_T1DEXI(subject_id):
    subject_cgm = read_preprocess_cgm_training(subject_id)
    interval_timedelta = datetime.timedelta(minutes=6)  # Example timedelta of 6 minutes, providing a range for latency

    # Create a list to store the results
    res = []

    # Initialize the first group
    if not subject_cgm.empty:
        current_group = [subject_cgm.iloc[0]['LBORRES']]
        last_time = subject_cgm.iloc[0]['LBDTC']

    # Iterate over rows in DataFrame starting from the second row
    for index, row in subject_cgm.iloc[1:].iterrows():
        current_time = row['LBDTC']
        if (current_time - last_time) <= interval_timedelta:
            # If the time difference is within the limit, add to the current group
            current_group.append(row['LBORRES'])
        else:
            # Otherwise, start a new group
            res.append(current_group)
            current_group = [row['LBORRES']]
        last_time = current_time

    # Add the last group if it's not empty
    if current_group:
        res.append(current_group)

    return res

# For loop to generate res for train and test

In [11]:
# Define the directory path
train_directory_path = r'C:\Users\baiyi\OneDrive\Desktop\BGprediction\LB_split\training'  # Use a raw string for paths on Windows

# List files without their extensions
train_file_names = [os.path.splitext(file)[0] for file in os.listdir(train_directory_path)
              if os.path.isfile(os.path.join(train_directory_path, file))]

# Print the list of file names
print(train_file_names)

['1343_training_data', '1617_training_data', '255_training_data', '354_training_data', '695_training_data', '816_training_data', '854_training_data', '856_training_data', '862_training_data', '894_training_data', '900_training_data', '907_training_data', '953_training_data', '979_training_data', '981_training_data', '987_training_data']


In [12]:
# Define the directory path
test_directory_path = r'C:\Users\baiyi\OneDrive\Desktop\BGprediction\LB_split\testing'  # Use a raw string for paths on Windows

# List files without their extensions
test_file_names = [os.path.splitext(file)[0] for file in os.listdir(test_directory_path)
              if os.path.isfile(os.path.join(test_directory_path, file))]

# Print the list of file names
print(test_file_names)

['1343_testing_data', '1617_testing_data', '255_testing_data', '354_testing_data', '695_testing_data', '816_testing_data', '854_testing_data', '856_testing_data', '862_testing_data', '894_testing_data', '900_testing_data', '907_testing_data', '953_testing_data', '979_testing_data', '981_testing_data', '987_testing_data']


In [13]:
train_data = dict()
for subj in train_file_names:
    print(subj)
    subj_path = f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/training/{subj}.csv'
    reader = preporcess_T1DEXI(subj_path)
    train_data[subj] = reader

1343_training_data
1617_training_data
255_training_data
354_training_data
695_training_data
816_training_data
854_training_data
856_training_data
862_training_data
894_training_data
900_training_data
907_training_data
953_training_data
979_training_data
981_training_data
987_training_data


In [14]:
# Have not been run
test_data = dict()
for subj in test_file_names:
    print(subj)
    subj_path = f'C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/testing/{subj}.csv'
    reader = preporcess_T1DEXI(subj_path)
    test_data[subj] = reader

1343_testing_data
1617_testing_data
255_testing_data
354_testing_data
695_testing_data
816_testing_data
854_testing_data
856_testing_data
862_testing_data
894_testing_data
900_testing_data
907_testing_data
953_testing_data
979_testing_data
981_testing_data
987_testing_data


In [15]:
epoch = 20

ph = 6 # Prediction horizon
path = "../t1dexi_results"

In [16]:
cleaned_subjects = [s.replace("_training_data", "") for s in train_file_names]
cleaned_subjects.sort()

In [19]:
# a dumb dataset instance
train_dataset = CGMSDataSeg(
    "ohio", "C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/training/255_training_data.csv", 5
)
sampling_horizon = 7
prediction_horizon = ph
scale = 0.01
outtype = "Same"
# train on training dataset
# k_size, nblock, nn_size, nn_layer, learning_rate, batch_size, epoch, beta
with open(f'../t1dexi_results/config.json') as json_file:
    config = json.load(json_file)
argv = (
    config["k_size"],
    config["nblock"],
    config["nn_size"],
    config["nn_layer"],
    config["learning_rate"],
    config["batch_size"],
    epoch,
    config["beta"],
)
l_type = config["loss"]
# test on patients data
outdir = os.path.join(path, f"ph_{prediction_horizon}_{l_type}")
if not os.path.exists(outdir):
    os.makedirs(outdir)
all_errs = []

2020-04-01 00:01:51
Reading 8 segments


In [20]:
# Loop
standard = False  # do not use standard
all_errs = []
for pid in overlap:
    # local_train_data = []
    
    # for k in train_data.keys():
    #     if k != pid:
    #         local_train_data += train_data[k]
    train_pids = set(overlap) - set([pid])
    local_train_data = []
    for k in train_pids:
        local_train_data += train_data[k + "_training_data"]
    print(f"Pretrain data: {sum([sum(x) for x in local_train_data])}")

    train_dataset.data = local_train_data
    train_dataset.set_cutpoint = -1
    train_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        100,
        False,
        outtype,
        1,
        standard,
    )
    regressor(train_dataset, *argv, l_type, outdir)
    # Fine-tune and test
    # target_test_dataset = CGMSDataSeg(
    #     "ohio", f"C:/Users/baiyi/OneDrive/Desktop/BGprediction/OhioT1DM/2018/test/{pid}-ws-testing.xml", 5
    # )
    target_test_dataset = CGMSDataSeg(
    "ohio", f"C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/testing/{pid}_testing_data.csv", 5
    )
    target_test_dataset.set_cutpoint = 1
    target_test_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        0.01,
        False,
        outtype,
        1,
        standard,
    )
    # target_train_dataset = CGMSDataSeg(
    #     "ohio", f"C:/Users/baiyi/OneDrive/Desktop/BGprediction/OhioT1DM/2018/test/{pid}-ws-testing.xml", 5
    # )
    target_train_dataset = CGMSDataSeg(
    "ohio", f"C:/Users/baiyi/OneDrive/Desktop/BGprediction/LB_split/training/{pid}_training_data.csv", 5
    )
    target_train_dataset.set_cutpoint = -1
    target_train_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        100,
        False,
        outtype,
        1,
        standard,
    )
    err, labels = test_ckpt(target_test_dataset, outdir)
    errs = [err]
    transfer_res = [labels]
    for i in range(1, 2):
        err, labels = regressor_transfer(
            target_train_dataset,
            target_test_dataset,
            config["batch_size"],
            epoch,
            outdir,
            i,
        )
        errs.append(err)
        transfer_res.append(labels)
    transfer_res = np.concatenate(transfer_res, axis=1)
    print(transfer_res)
    np.savetxt(
        f"{outdir}/{pid}.txt",
        transfer_res,
        fmt="%.4f %.4f %.4f %.4f", #%.4f %.4f %.4f %.4f
    )
    all_errs.append([pid] + errs)
all_errs = np.array(all_errs)
np.savetxt(f"{outdir}/errors.txt", all_errs, fmt="%s %.4f %.4f") # %.4f %.4f
# label pair:(groundTruth, y_pred)


Pretrain data: 12359130.0
Building dataset, requesting data from 0 to 294
Train data requested beyond limit, using all but last one
############################ Data structure summary ############################
Hypo/no_hypo: 6458/80116
Found 294 continuous time series
Data shape: (86576, 7), Train/test: 86574/2
Train test ratio: 43287.00
################################################################################
Feature size is: 
0
In regressor, x =
Tensor("x:0", shape=(None, 7), dtype=float32)
In regressor, y =
Tensor("add:0", shape=(None, 7), dtype=float32)
line73: Shape of y: (None, 7)
Before L2 regularization
Before training for loop
int(low_fid_data.train_n / batch_size) =  1352
Epoch 0, train loss: 0.275283
Epoch 1, train loss: 0.199644
Epoch 2, train loss: 0.212992
Epoch 3, train loss: 0.174614
Epoch 4, train loss: 0.182365
Epoch 5, train loss: 0.206565
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch 6, train loss: 0.236761
Epoch 

TypeError: Mismatch between array dtype ('<U32') and format specifier ('%s %.4f %.4f')

In [ ]:
['854', '0.15074661', '0.14032947'],
['979', '0.10964842', '0.1068475'],
['816', '0.17296007', '0.15637168'],
['953', '0.16875254', '0.16648282'],
['981', '0.18782552', '0.1781756'],
['1617', '0.18750052', '0.17405139'],
['1343', '0.1479887', '0.12966709'],
['987', '0.1956755', '0.18187545'],
['255', '0.112035595', '0.11363354'],
['907', '0.2046883', '0.18899176'],
['856', '0.16526204', '0.15994464'],
['354', '0.21569382', '0.21845779'],
['894', '0.1430959', '0.14199'],
['862', '0.2226933', '0.19469088'],
['900', '0.14939056', '0.12725726'],
['695', '0.16643564', '0.16699977']

In [21]:
all_errs

array([['854', '0.15074661', '0.14032947'],
       ['979', '0.10964842', '0.1068475'],
       ['816', '0.17296007', '0.15637168'],
       ['953', '0.16875254', '0.16648282'],
       ['981', '0.18782552', '0.1781756'],
       ['1617', '0.18750052', '0.17405139'],
       ['1343', '0.1479887', '0.12966709'],
       ['987', '0.1956755', '0.18187545'],
       ['255', '0.112035595', '0.11363354'],
       ['907', '0.2046883', '0.18899176'],
       ['856', '0.16526204', '0.15994464'],
       ['354', '0.21569382', '0.21845779'],
       ['894', '0.1430959', '0.14199'],
       ['862', '0.2226933', '0.19469088'],
       ['900', '0.14939056', '0.12725726'],
       ['695', '0.16643564', '0.16699977']], dtype='<U32')

In [22]:

# Convert the second and third columns to floats
second_column = all_errs[:, 1].astype(float)
third_column = all_errs[:, 2].astype(float)

# Calculate the average
average_second_column = np.mean(second_column)
average_third_column = np.mean(third_column)

print("Average of the second column:", average_second_column)
print("Average of the third column:", average_third_column)

Average of the second column: 0.16877456468750002
Average of the third column: 0.159110415


In [ ]:
['854', '0.14176467', '0.13840644'],
['979', '0.11082648', '0.10547459'],
['816', '0.17233326', '0.15507208'],
['953', '0.16803236', '0.15838838'],
['981', '0.18506993', '0.17628354'],
['1617', '0.1810529', '0.17188448'],
['1343', '0.14670542', '0.12550768'],
['987', '0.1974195', '0.18113053'],
['255', '0.11360771', '0.113014214'],
['85', '0.24868742', '0.23800832'],
['907', '0.198933', '0.1871577'],
['856', '0.1641185', '0.16184378'],
['354', '0.21777242', '0.22120297'],
['894', '0.14986292', '0.1430042'],
['911', '0.3157179', '0.30788115'],
['862', '0.20412911', '0.19469687'],
['900', '0.14108652', '0.1235286'],
['695', '0.18116617', '0.16922566'] ## 30MIN

In [2]:
import numpy as np

all_errs = np.array([['854', '0.14176467', '0.13840644'],
['979', '0.11082648', '0.10547459'],
['816', '0.17233326', '0.15507208'],
['953', '0.16803236', '0.15838838'],
['981', '0.18506993', '0.17628354'],
['1617', '0.1810529', '0.17188448'],
['1343', '0.14670542', '0.12550768'],
['987', '0.1974195', '0.18113053'],
['255', '0.11360771', '0.113014214'],
['907', '0.198933', '0.1871577'],
['856', '0.1641185', '0.16184378'],
['354', '0.21777242', '0.22120297'],
['894', '0.14986292', '0.1430042'],
['862', '0.20412911', '0.19469687'],
['900', '0.14108652', '0.1235286'],
['695', '0.18116617', '0.16922566']])

In [3]:
# Convert the second and third columns to floats
second_column = all_errs[:, 1].astype(float)
third_column = all_errs[:, 2].astype(float)

# Calculate the average
average_second_column = np.mean(second_column)
average_third_column = np.mean(third_column)

print("Average of the second column:", average_second_column)
print("Average of the third column:", average_third_column)

Average of the second column: 0.16711755437499998
Average of the third column: 0.157863857125


 The altimate result please go into the result folder to check the evaluation.ipynb

In [ ]:
# If run into an error, but all result txt files are ready, please run this section for evaluation
import argparse
import json
import os
import pandas as pd
import os
import datetime
import numpy as np

# List all files and directories in the current directory
files_and_directories = os.listdir('./t1dexi_results')

# Filter for files that end with .txt
txt_files = [file for file in files_and_directories if file.endswith('.txt')]

# Read the data from the text file
def calcuate_rmse(file):
    data = np.loadtxt(file)  # Make sure to replace 'data.txt' with your actual file path
    print(file)
    # Splitting the data into groundtruth and predictions
    groundtruth = data[:, 0]  # First column as ground truth (also same as third column)
    predictions_1 = data[:, 1]  # Second column as predictions from method 1
    predictions_2 = data[:, 3]  # Fourth column as predictions from method 2

    # Function to calculate RMSE
    def calculate_rmse(true_values, predictions):
        mse = np.mean((true_values - predictions) ** 2)
        rmse = np.sqrt(mse)
        return rmse

    # Calculate RMSE for each method
    rmse_method_1 = calculate_rmse(groundtruth, predictions_1)
    rmse_method_2 = calculate_rmse(groundtruth, predictions_2)

    print("RMSE for Method 1:", rmse_method_1)
    print("RMSE for Method 2:", rmse_method_2)
    return rmse_method_1


rmse_list = []
for f in txt_files[:-1]:
    rmse1 = calcuate_rmse(f)
    print(rmse1)
    rmse_list.append(rmse1)

print(np.average(rmse_list))


In [ ]:
rmse_list